In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [56]:
df = pd.read_csv('lamudi_malang_house.csv', encoding='ISO-8859-1')
df.head()

,title,location,detail,price,agent_name,agent_status,description
0,Rumah mewah termurah dekat kampus brawijaya,"Dinoyo, Malang",3 Kamar tidur ...,Rp662.000.000,Endang AJ Property,Rekan Lamudi Pro,MODERN RUMAH READY UNIT 2 LANTAI mulai 600juta...
1,Di jual Rumah Kost Exclusive Kota Malang Suhat,"Tulusrejo, Malang",10 Kamar tidur ...,Rp2.500.000.000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Redy Redy 2 U...
2,di jual rumah kost Exclusive kota malang Suhat...,"Lowokwaru, Malang",10 Kamar tidur ...,Rp2.900.000.000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Sisa 1 unit S...
3,Dijual Murah Rumah Aesthetic di Lowokwaru Deka...,"Lowokwaru, Malang",2 Kamar tidur ...,Rp589.000.000,Muhammad Sudono,Agent,Akses mobil bisa simpangan .Bisa KPR .Row jala...
4,Dijual Rumah Butuh uang segera luas 231m di p...,"Blimbing, Malang",3 Kamar tidur ...,Rp650.000.000,Fran Jual Rumah Malang,Rekan Lamudi Pro,Dijual Rumah luas 231m butuh uang segera lokas...


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2958 entries, 0 to 2957
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         2958 non-null   object
 1   location      2958 non-null   object
 2   detail        2958 non-null   object
 3   price         2958 non-null   object
 4   agent_name    2958 non-null   object
 5   agent_status  2958 non-null   object
 6   description   2958 non-null   object
dtypes: object(7)
memory usage: 161.9+ KB


## Cleaning Data   

#### Memecah location menjadi Kabupaten dan Kota

In [57]:
df[['kabupaten', 'kota']] = df['location'].str.split(r',', expand=True)
df = df.drop(['location'], axis=1)
df.head()

,title,detail,price,agent_name,agent_status,description,kabupaten,kota
0,Rumah mewah termurah dekat kampus brawijaya,3 Kamar tidur ...,Rp662.000.000,Endang AJ Property,Rekan Lamudi Pro,MODERN RUMAH READY UNIT 2 LANTAI mulai 600juta...,Dinoyo,Malang
1,Di jual Rumah Kost Exclusive Kota Malang Suhat,10 Kamar tidur ...,Rp2.500.000.000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Redy Redy 2 U...,Tulusrejo,Malang
2,di jual rumah kost Exclusive kota malang Suhat...,10 Kamar tidur ...,Rp2.900.000.000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Sisa 1 unit S...,Lowokwaru,Malang
3,Dijual Murah Rumah Aesthetic di Lowokwaru Deka...,2 Kamar tidur ...,Rp589.000.000,Muhammad Sudono,Agent,Akses mobil bisa simpangan .Bisa KPR .Row jala...,Lowokwaru,Malang
4,Dijual Rumah Butuh uang segera luas 231m di p...,3 Kamar tidur ...,Rp650.000.000,Fran Jual Rumah Malang,Rekan Lamudi Pro,Dijual Rumah luas 231m butuh uang segera lokas...,Blimbing,Malang


In [59]:
def cleaning(text):
    text = re.sub(r'[.,?]', ' ', text)
    text = re.sub(r'[Rp, m²]', ' ', text)
    text = re.sub(r'\s+', '', text)
    return text

In [60]:
df['price'] = df['price'].apply(cleaning)
df['detail'] = df['detail'].apply(cleaning)
df.head()

,title,detail,price,agent_name,agent_status,description,kabupaten,kota
0,Rumah mewah termurah dekat kampus brawijaya,3Kaartidur60ÂBangunan69ÂLahan,662000000,Endang AJ Property,Rekan Lamudi Pro,MODERN RUMAH READY UNIT 2 LANTAI mulai 600juta...,Dinoyo,Malang
1,Di jual Rumah Kost Exclusive Kota Malang Suhat,10Kaartidur250ÂBangunan103ÂLahan,2500000000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Redy Redy 2 U...,Tulusrejo,Malang
2,di jual rumah kost Exclusive kota malang Suhat...,10Kaartidur300ÂBangunan103ÂLahan,2900000000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Sisa 1 unit S...,Lowokwaru,Malang
3,Dijual Murah Rumah Aesthetic di Lowokwaru Deka...,2Kaartidur36ÂBangunan72ÂLahan,589000000,Muhammad Sudono,Agent,Akses mobil bisa simpangan .Bisa KPR .Row jala...,Lowokwaru,Malang
4,Dijual Rumah Butuh uang segera luas 231m di p...,3Kaartidur231ÂBangunan231ÂLahan,650000000,Fran Jual Rumah Malang,Rekan Lamudi Pro,Dijual Rumah luas 231m butuh uang segera lokas...,Blimbing,Malang


In [37]:
df_backup.head()

,title,detail,price,agent_name,agent_status,description,Kabupaten,Kota
0,Rumah mewah termurah dekat kampus brawijaya,3 Kamar tidur ...,Rp662.000.000,Endang AJ Property,Rekan Lamudi Pro,MODERN RUMAH READY UNIT 2 LANTAI mulai 600juta...,Dinoyo,Malang
1,Di jual Rumah Kost Exclusive Kota Malang Suhat,10 Kamar tidur ...,Rp2.500.000.000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Redy Redy 2 U...,Tulusrejo,Malang
2,di jual rumah kost Exclusive kota malang Suhat...,10 Kamar tidur ...,Rp2.900.000.000,Arif Faisol Halim,Rekan Lamudi,Di Jual Rumah kost Exclusive SHM Sisa 1 unit S...,Lowokwaru,Malang
3,Dijual Murah Rumah Aesthetic di Lowokwaru Deka...,2 Kamar tidur ...,Rp589.000.000,Muhammad Sudono,Agent,Akses mobil bisa simpangan .Bisa KPR .Row jala...,Lowokwaru,Malang
4,Dijual Rumah Butuh uang segera luas 231m di p...,3 Kamar tidur ...,Rp650.000.000,Fran Jual Rumah Malang,Rekan Lamudi Pro,Dijual Rumah luas 231m butuh uang segera lokas...,Blimbing,Malang


In [13]:
df['agent_name'].value_counts()

agent_name
Iva Fiatur                               114
Xavier Marks Premier Malang - Hwa Hwa    113
Sastro .                                 107
HRP - Habibur Rohman                      92
Myuu Naa                                  75
                                        ... 
Lilik S                                    1
Bobby Ovan                                 1
Eva Y                                      1
Jason Liem                                 1
Wachid H                                   1
Name: count, Length: 309, dtype: int64

In [33]:
df['detail'].values

array(['3                                Kamar tidur                                    60 mÂ²                                Bangunan                                     69 mÂ²                                Lahan',
       '10                                Kamar tidur                                    250 mÂ²                                Bangunan                                     103 mÂ²                                Lahan',
       '10                                Kamar tidur                                    300 mÂ²                                Bangunan                                     103 mÂ²                                Lahan',
       ...,
       '2                                Kamar tidur                                    25 mÂ²                                Bangunan                                     60 mÂ²                                Lahan',
       '2                                Kamar tidur                                    25 mÂ²                    

In [31]:
df_split = df['detail'].str.split('|', expand=True)

df_update = pd.concat([df, df_split], axis=1)

kolom_baru = [f'Nilai_{i+1}' for i in range(df_split.shape[1])]
df_update.columns = kolom_baru

print(df_update)

ValueError: Length mismatch: Expected axis has 9 elements, new values have 1 elements

0    3                                Kamar tidur  ...
1    10                                Kamar tidur ...
2    10                                Kamar tidur ...
3    2                                Kamar tidur  ...
4    3                                Kamar tidur  ...
Name: detail, dtype: object